# Pythonのforループ実行時間を正確に測りたい

まず、講義資料で紹介されている下記のコードを検証しました。

In [1]:
!time
a = 0
for i in range(10000):
  a += i**2
a


real	0m0.000s
user	0m0.000s
sys	0m0.000s


333283335000

講義中、この結果からPythonだとループの計算に0.000秒しか時間がかかっていないと紹介されました。しかし、このコードはセル内の実行時間を計測しておりません。セルの２行目以下のPythonコードは確かに実行されてはいますが、その実行時間が表示されているわけではありません。もし、実行時間が計測できているのであれば、実行結果の前に実行時間が表示されているのはおかしいからです。また、実際にこのコードで行われていることは、Jupyterにおいては、!で始まるコマンドはOSコマンドの呼び出しであり、ここではLinux（Google Colabの中身はUbuntu）のtimeコマンドを実行しています。そして、そのtimeコマンドの後ろに計測すべきLinuxコマンドが書かれていないため、実行時間が０になっています。（余談ですが、ちなみに、!time コマンドの使い道としてはたとえば、!time ls などとするとlsコマンドの実行時間を計測できます。）

Jupyterで実行時間を計測するためのマジックコマンドは、１行のみのセルの場合は%time、複数行のセルの場合は、%%timeです。そのため、今回のforループを１回実行したときの実行時間を計測するスクリプトは以下のようになります。

In [2]:
%%time
a = 0
for i in range(10000):
  a += i**2
print(a)

333283335000
CPU times: user 6.57 ms, sys: 118 µs, total: 6.69 ms
Wall time: 6.49 ms


冒頭のコードと違って、実行結果を表示した後に、実行時間が表示されていることがわかります。print()自体に時間がかかっている可能性を排除するためには、下記のコードを検討するとよさそうです。

In [3]:
%%time
a = 0
for i in range(10000):
  a += i**2
a

CPU times: user 5.87 ms, sys: 0 ns, total: 5.87 ms
Wall time: 5.72 ms


In [4]:
print(a)

333283335000


マジックコマンドではなく、time モジュールを使って処理の前後の時間の差分を使って実行時間を測る手法もあります。

In [5]:
import time
start = time.time()
a = 0
for i in range(10000):
  a += i**2
a
e_time = time.time() - start
print ("e_time:{0}".format(e_time) + "[s]")

e_time:0.010224342346191406[s]


ただし、１回のみの実行だけだとどれくらいばらつくかがわからないので、実行時間をより正確に測るには複数回繰り返して測定して、分布を見るのがよいでしょう。その目的のために使えるマジックコマンドが %%timeit です。

In [6]:
%%timeit
a = 0
for i in range(10000):
  a += i**2
a

100 loops, best of 5: 3.07 ms per loop


%%timeit は -n や -r に続けて数値を指定すると、それぞれnumberとrepeatの回数を指定もできます。cf. https://note.nkmk.me/en/python-timeit-measure/

Pythonのforループで10000個の平方数の和をGoogle Colab上で計算するのに、100回（loop）を５回（repeat）繰り返して、５回中一番早かったときの平均で3.07 msかかっていることがわかります（バックグラウンド処理のノイズなどの影響を取り除くために、このような表示がされるようです）。

# Rのforループがどれくらいかかるのか知りたい

ノートブックをRカーネルで別途作ってもよいですが、ここでは同じファイル内で結果をまとめたいので、rpy2を用いてPythonカーネル内でRを実行できるようにする。

In [7]:
%load_ext rpy2.ipython

上記のセルを実行した後、セルの先頭にに%%Rのマジックコマンドをつけると、そのセルはRのコードとして認識され、実行されます。（コードが１行飲みの場合は、%R の後ろにRのコードを書いてもよいです。）

In [ ]:
%%R
install.packages("tictoc")
library(tictoc)

In [9]:
%%R
tic()
a <- 0
for(i in 1:10000){
    a <- a + i^2
  }
toc()

0.082 sec elapsed


tictocパッケージで計測した１回の実行時間は0.1秒以下くらいのようです。実行するタイミングにもよるかもしれません。Pythonのときと同様、何度も繰り返した場合にどれくらいばらつくかも知りたいので、実行時間の平均や分布を見ることにします。microbenchmarkパッケージが比較するのに便利です。

In [ ]:
%%R
install.packages("microbenchmark")

In [11]:
%%R
library(microbenchmark)

In [12]:
%%R
naive_for_loop <- function(n){
  for(i in 1:n){
    a <- 0
    a <- a + i^2
  }
}

naive_for_loop2 <- function(n){
  a <- 0
  for(i in 1:n){
    a <- a + i^2
  }
}

func_square <- function(x){
  x^2
}

use_sapply1 <- function(n){
  a <- 1:n
  sum(sapply(a, func_square))
}

use_sapply2 <- function(n){
  a <- 1:n
  sum(sapply(a, \(x) x^2))
}

func_sum_square <- function(n){
  a <- 1:n
  sum(n^2)
}

In [13]:
%%R
results <- microbenchmark(
  naive_for_loop(10000), 
  naive_for_loop2(10000), 
  use_sapply1(10000), 
  use_sapply2(10000),
  func_sum_square(10000), 
  times = 100, 
  unit = "s"
  )

In [14]:
%R summary(results)

,expr,min,lq,mean,median,uq,max,neval
1,naive_for_loop(10000),4.617860e-04,0.000471,0.000564,0.000481,0.000495,0.004543,100.0
2,naive_for_loop2(10000),3.500300e-04,0.000357,0.000436,0.000364,0.000376,0.003715,100.0
3,use_sapply1(10000),6.460866e-03,0.006725,0.007765,0.007128,0.007782,0.017343,100.0
4,use_sapply2(10000),6.464354e-03,0.006798,0.008105,0.007403,0.008934,0.012469,100.0
5,func_sum_square(10000),8.550000e-07,0.000002,0.000030,0.000005,0.000006,0.002500,100.0


いろいろな書き方で比較してみました。一番厳しい比較として、100回繰り返したときの最悪計算時間（max）で比較したとしても、Rで書いたナイーブなforループは１回の実行時間が3.7〜4.5msec程度で、冒頭で紹介したPythonのforループと大差ないようです。また、書き方の工夫が足りないのかapply()系のメリット（高速化）もこの場合はあまり感じられません（計測の仕方との相性が悪いのかもしれません）。ただし、ベクトル化した一番最後の方法では平均や中央値で見ても爆速になっていますので、大量の計算を行う場合は書き方に気をつけると時短が望めるかもしれません。

# まとめると、Rはとにかくforループが遅い（Pythonと比較してかなり遅い）というのは、最近のRにおいては事実と異なるので訂正が必要です。

というのも、R 3.4.0以降はJITコンパイルがデフォルトで取り入れられたため、forループやfunctionが遅かったかつてのRほどは問題になりにくくなったからです。ただし、どの言語でも書き方を工夫すればちゃんと速くなるので、気をつけるに越したことはありません。